# Training CGNet

Purpose:
--------
The purpose of this notebook is to train CGnet for machine learning detection of atmospheric rivers and tropical cyclones.\
See ClimateNet repo here: https://github.com/andregraubner/ClimateNet

Authors/Contributors:
---------------------
* Teagan King
* John Truesdale
* Katie Dagon

## Import libraries

In [1]:
import os
import sys
import json
import numpy as np

sys.path.append("/glade/work/kdagon/ClimateNet") # append path to ClimateNet repo
from climatenet.utils.data import ClimateDatasetLabeled, ClimateDataset
from climatenet.models import CGNet
from climatenet.utils.utils import Config
from climatenet.track_events import track_events
from climatenet.analyze_events import analyze_events
from climatenet.visualize_events import visualize_events

from os import path

## Config file
Use `get_averages_and_standard_devs.ipynb` to calculate means/stds for given training dataset.

In [2]:
#config = Config('/glade/work/kdagon/ML-extremes/trained_models/config_021523.json') # weighted mean, unweighted std
config = Config('/glade/work/kdagon/ML-extremes/trained_models/config_040523.json') # unweighted mean and std

In [3]:
config.description

'The basic CGNet model. You can use this config to train your own model, or load it with our trained weights.'

In [4]:
config.train_batch_size

16

## Confirm GPU resources
Can request through JupyterHub launch page.\
Current resources request (2/15/23): 1 node, 4 cpu, 64GB mem, 2 V100 GPU\
4/5/23: Noticed that not all CPU/mem was needed, new request: 1 node, 2 cpu, 64GB mem, 2 V100 GPU

In [5]:
# requires loading pytorch into environment
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
2


## Instantiate CGNet model given config file
This step needs GPU resources (as written)

In [6]:
%%time
cgnet = CGNet(config)

CPU times: user 1.44 s, sys: 708 ms, total: 2.14 s
Wall time: 2.15 s


In [7]:
cgnet.optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
)

## Set train, test data

In [8]:
train_path = "/glade/campaign/cgd/amp/jet/ClimateNet_12012020/portal.nersc.gov/project/ClimateNet/climatenet_new"

train = ClimateDatasetLabeled(path.join(train_path, 'train'), config)
test = ClimateDatasetLabeled(path.join(train_path, 'test'), config)

In [9]:
train.fields

{'TMQ': {'mean': 19.218494415283203, 'std': 15.817276954650879},
 'U850': {'mean': 1.5530235767364502, 'std': 8.29762077331543},
 'V850': {'mean': 0.2541317045688629, 'std': 6.231630802154541},
 'PSL': {'mean': 100814.078125, 'std': 1461.225830078125}}

In [10]:
train.length

398

In [13]:
config.epochs

20

## Train model

Memory use holding at ~14GB during training\
Each epoch takes ~1 min to run

In [11]:
cgnet.train(train) # use ~20 epochs for non-test
# IOU mean should be around 0.75 after all epochs?
# weights and measures site to look at ML performance
# maybe playing with those parameters could improve model

  0%|          | 0/25 [00:00<?, ?it/s]

Epoch 1:


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch stats:
[[2.78531995e+08 6.72942200e+06 4.52592820e+07]
 [1.08767700e+06 2.55390000e+05 2.83280000e+05]
 [6.13705900e+06 2.86185000e+05 1.35546380e+07]]
IOUs:  [0.82468027 0.02955234 0.20687647] , mean:  0.3537030257391982
Epoch 2:


  0%|          | 0/25 [00:00<?, ?it/s]

...

Epoch stats:
[[3.18562647e+08 1.05667800e+06 1.09013740e+07]
 [5.25473000e+05 1.02087200e+06 8.00020000e+04]
 [5.33830800e+06 8.21470000e+04 1.45574270e+07]]
IOUs:  [0.94701946 0.36918933 0.4702124 ] , mean:  0.5954737285829655
Epoch 19:


  0%|          | 0/25 [00:00<?, ?it/s]

Epoch stats:
[[3.18584008e+08 1.10457800e+06 1.08321130e+07]
 [5.07651000e+05 1.03895200e+06 7.97440000e+04]
 [5.19961500e+06 7.90610000e+04 1.46992060e+07]]
IOUs:  [0.94752383 0.36973565 0.47586048] , mean:  0.597706651871503
Epoch 20:


Loss: 0.7792255878448486: 100%|██████████| 25/25 [01:07<00:00,  2.69s/it]

Epoch stats:
[[3.18784066e+08 1.04001400e+06 1.06966190e+07]
 [5.20870000e+05 1.02646100e+06 7.90160000e+04]
 [5.22794200e+06 7.90720000e+04 1.46708680e+07]]
IOUs:  [0.94800169 0.37387946 0.47704684] , mean:  0.5996426625629762


2/15/23: Training is maxing out around 0.5977 IOU mean at last epoch\
4/5/23: With updated means, similar performance: 0.5997 IOU mean at last epoch

### TO DO: output masks during training so we can visually check match between training data and what model predicts?
Instead of simply relying on IOU. Also check the loss function here.

### TO DO: increase # epochs? play with batch size, other hyperparameters
See CGnet papers (Kapp-Schwoerer et al. and Lacombe et al.)

## Evaluate model on test data

In [14]:
test.fields

{'TMQ': {'mean': 19.218494415283203, 'std': 15.817276954650879},
 'U850': {'mean': 1.5530235767364502, 'std': 8.29762077331543},
 'V850': {'mean': 0.2541317045688629, 'std': 6.231630802154541},
 'PSL': {'mean': 100814.078125, 'std': 1461.225830078125}}

In [15]:
test.length

61

In [16]:
%%time
cgnet.evaluate(test)

100%|██████████| 4/4 [00:09<00:00,  2.49s/it]

Evaluation stats:
[[4.9207437e+07 1.6860800e+05 1.2782570e+06]
 [1.2489300e+05 1.6929100e+05 2.4100000e+03]
 [1.2919190e+06 1.5389000e+04 1.7106920e+06]]
IOUs:  [0.9450045  0.35225587 0.39795872] , mean:  0.5650730277914003
CPU times: user 1.64 s, sys: 1.65 s, total: 3.29 s
Wall time: 9.97 s


2/15/23: Testing data is at 0.55 IOU mean\
4/5/23: Testing data is at 0.56 IOU mean

## Save out model

In [17]:
# makes a folder by the name specified, puts the trained model and config file in there
#cgnet.save_model('/glade/work/kdagon/ML-extremes/trained_models/trained_cgnet.021523')
cgnet.save_model('/glade/work/kdagon/ML-extremes/trained_models/trained_cgnet.040523')

In [ ]:
# this one way to load in previously trained model
#cgnet.load_model('/glade/work/kdagon/ML-extremes/trained_models/trained_cgnet.021523')

## Set inference data

In [38]:
year=2000
#inference_path = "/glade/campaign/cgd/amp/jet/ClimateNet/Climate_data_"+str(year)
#inference_path = "/glade/scratch/tking/cgnet/historical_2000_2005/split_files"
inference_path = "/glade/campaign/cgd/ccr/kdagon/cgnet/B20TRC5CN/"+str(year)

inference = ClimateDataset(inference_path, config)  # could test different config with std/means for inference data

In [39]:
inference.fields

{'TMQ': {'mean': 24.927238169017997, 'std': 15.817276954650879},
 'U850': {'mean': 1.0356735863118816, 'std': 8.29762077331543},
 'V850': {'mean': 0.20847854977498861, 'std': 6.231630802154541},
 'PSL': {'mean': 101095.03520124489, 'std': 1461.225830078125}}

In [40]:
inference.length

17520

## Inference mode

In [35]:
%%time
class_masks = cgnet.predict(inference) # masks with 1==TC, 2==AR

100%|██████████| 183/183 [06:38<00:00,  2.18s/it]


CPU times: user 2min 17s, sys: 1min 8s, total: 3min 26s
Wall time: 6min 46s


## Save out masks

In [36]:
%%time
class_masks.to_netcdf("/glade/scratch/kdagon/cgnet/class_masks."+str(year)+".nc")

CPU times: user 55.9 ms, sys: 4.16 s, total: 4.22 s
Wall time: 7.53 s


## Track events
Create masks with event IDs

Note: memory spikes here - resource intensive!

In [ ]:
%%time
event_masks = track_events(class_masks)

In [ ]:
%%time
event_masks.to_netcdf("/glade/scratch/kdagon/cgnet/event_masks."+str(year)+".test.nc")

## Analyze events
Produce some visualizations

In [ ]:
analyze_events(event_masks, class_masks, "/glade/scratch/kdagon/cgnet/")

In [ ]:
visualize_events(event_masks, inference, "/glade/scratch/kdagon/cgnet/")   

## Weights and Biases
https://docs.wandb.ai/quickstart \
Still figuring out how this works...

In [42]:
import wandb

In [44]:
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········


wandb: Appending key for api.wandb.ai to your netrc file: /glade/u/home/kdagon/.netrc


True

In [46]:
wandb.init(project="climatenet-test", entity="katie-dagon")

Ignoring settings passed to wandb.setup() which has already been configured.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········


wandb: Appending key for api.wandb.ai to your netrc file: /glade/u/home/kdagon/.netrc
wandb: wandb version 0.13.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20230215_172859-2auiboc1
wandb: Syncing run decent-river-1


In [47]:
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 20,
  "batch_size": 16
}

In [49]:
wandb.log({"loss": loss})

NameError: name 'loss' is not defined